# Berechne Metriken für Validation- und Testdaten mit dem YOLOv8 Modell

In [21]:
import dill
import numpy as np
from ultralytics import YOLO

# Function

In [19]:
def createDictionary(metrics):
  """ convert the ultralytics metric object to dictionary """
  dct_metrics = {}
  dct_metrics['all_ap'] = metrics.box.all_ap # mAP für jede Klasse mit IoU Schritten von 0.5 bis 0.95 (0.05 Schritte)
  dct_metrics['ap'] = metrics.box.ap
  dct_metrics['ap50'] = metrics.box.ap50
  dct_metrics['ap_class_index'] = metrics.box.ap_class_index
  dct_metrics['f1'] = metrics.box.f1
  dct_metrics['map'] = metrics.box.map
  dct_metrics['map50'] = metrics.box.map50
  dct_metrics['map75'] = metrics.box.map75
  dct_metrics['maps'] = metrics.box.maps
  dct_metrics['mp'] = metrics.box.mp
  dct_metrics['mr'] = metrics.box.mr
  dct_metrics['nc'] = metrics.box.nc
  dct_metrics['p'] = metrics.box.p
  dct_metrics['r'] = metrics.box.r
  return dct_metrics


In [59]:
def calculateMetric(metrics):
    """
    Berechnet Metriken für alle Klassen und spezifisch für die Klassen einzeln
    
    Parameters
    ----------
    metrics : dict       Dictionary mit Metriken

    Returns
    -------
    None

    Gibt die Metriken für die gegebenen Klassen in Konsole aus
    """
    clsNames = ['car', 'pedestrian', 'trafficLight', 'truck', 'biker']

    print("Alle Klassen:")
    print("mAP@0.5:0.95:\t", round(metrics['map'], 3))
    print("mAP@0.5:\t", round(metrics['map50'], 3))
    print("mAP@0.75:\t", round(metrics['map75'], 3))

    for clsID in range(metrics['nc']):
        print("\nKlasse ", clsID, " (", clsNames[clsID], "):")
        print("mAP@0.5:0.95:\t", round(metrics['all_ap'][clsID].mean(), 3))
        print("mAP@0.5:\t", round(metrics['all_ap'][clsID, 0], 3))
        print("mAP@0.75:\t", round(metrics['all_ap'][clsID, 5], 3))



## Lade Modell

In [ ]:
# Load model
model = YOLO('runs/detect/yolov8m_200_epoch/weights/best.pt')

## Metriken für Validation

In [16]:
# Validate model
metrics = model.val(batch=8, imgsz=512, iou=0.6, conf=0.001, split="val")
dct_metrics = createDictionary(metrics)
with open('./yolov8Metrics/metrics_val.pkl', 'wb') as file:
    dill.dump(dct_metrics, file)

Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
val: Scanning /content/drive/MyDrive/Deep Vision/yolov8Model/datasets/dataYolov8/val/labels.cache... 2661 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2661/2661 [00:00<?, ?it/s]
val: WARNING ⚠️ /content/drive/MyDrive/Deep Vision/yolov8Model/datasets/dataYolov8/val/images/1478899459750057064_jpg.rf.0458fc6b3f19834e7e3054789b3b3873.jpg: 1 duplicate labels removed
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 333/333 [01:06<00:00,  5.00it/s]
                   all       2661      19808      0.858       0.73      0.793      0.501
                   car       2661      12949      0.884      0.825      0.873      0.607
            pedestrian       2661       2375      0.796       0.59       0.67      0.342
          trafficLight       2661       3422      0.902      0.767      0.831      0.499
                 truck       2661        691  

In [22]:
# Lade Metriken
with open('./yolov8Metrics/metrics_val.pkl', 'rb') as file:
    metrics = dill.load(file)

In [60]:
calculateMetric(metrics)

Alle Klassen:
mAP@0.5:0.95:	 0.501
mAP@0.5:	 0.793
mAP@0.75:	 0.528

Klasse  0  ( car ):
mAP@0.5:0.95:	 0.607
mAP@0.5:	 0.873
mAP@0.75:	 0.684

Klasse  1  ( pedestrian ):
mAP@0.5:0.95:	 0.342
mAP@0.5:	 0.67
mAP@0.75:	 0.297

Klasse  2  ( trafficLight ):
mAP@0.5:0.95:	 0.499
mAP@0.5:	 0.831
mAP@0.75:	 0.501

Klasse  3  ( truck ):
mAP@0.5:0.95:	 0.646
mAP@0.5:	 0.877
mAP@0.75:	 0.743

Klasse  4  ( biker ):
mAP@0.5:0.95:	 0.412
mAP@0.5:	 0.713
mAP@0.75:	 0.416


## Metriken für Test

In [17]:
# Validate model
metrics = model.val(batch=8, imgsz=512, iou=0.6, conf=0.001, split="test")
dct_metrics = createDictionary(metrics)
with open('./yolov8Metrics/metrics_test.pkl', 'wb') as file:
    dill.dump(dct_metrics, file)


Ultralytics YOLOv8.0.136 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
val: Scanning /content/drive/MyDrive/Deep Vision/yolov8Model/datasets/dataYolov8/test/labels.cache... 1311 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1311/1311 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 164/164 [00:48<00:00,  3.36it/s]
                   all       1311       9553      0.859       0.73      0.794      0.498
                   car       1311       6284      0.886      0.823      0.873      0.609
            pedestrian       1311        986      0.807      0.626      0.703       0.36
          trafficLight       1311       1773      0.896      0.743      0.817      0.493
                 truck       1311        325      0.841      0.796      0.835      0.612
                 biker       1311        185      0.867      0.665      0.741      0.414
Speed: 0.3ms preprocess, 18.1ms inference, 0.0ms loss

In [61]:
# Lade Metriken
with open('./yolov8Metrics/metrics_test.pkl', 'rb') as file:
    metrics = dill.load(file)

## Metriken für Test

In [62]:
calculateMetric(metrics)

Alle Klassen:
mAP@0.5:0.95:	 0.498
mAP@0.5:	 0.794
mAP@0.75:	 0.523

Klasse  0  ( car ):
mAP@0.5:0.95:	 0.609
mAP@0.5:	 0.873
mAP@0.75:	 0.691

Klasse  1  ( pedestrian ):
mAP@0.5:0.95:	 0.36
mAP@0.5:	 0.703
mAP@0.75:	 0.316

Klasse  2  ( trafficLight ):
mAP@0.5:0.95:	 0.493
mAP@0.5:	 0.817
mAP@0.75:	 0.505

Klasse  3  ( truck ):
mAP@0.5:0.95:	 0.612
mAP@0.5:	 0.835
mAP@0.75:	 0.696

Klasse  4  ( biker ):
mAP@0.5:0.95:	 0.414
mAP@0.5:	 0.741
mAP@0.75:	 0.406
